In [1]:
import sys
import os
root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(root_path)
import json
import logging

from typing import List, Tuple, Dict
from src.common.prompts.value_adding_analysis.gpt_components import VAPromptComponents

ROOT_DIR = r"C:\Projects\Research\SWEEP\SWEEP"

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_path)

from src.common.models import ActivityBreakdown, ValueAddingAnalysis
from src.value_adding_analysis import ValueClassificationComponentsGPT, compare_value_classifications, print_comparison_results
from src.common.util.utils import get_activities, get_sectors

def get_file_paths(sector: str, activity: str, model_name: str, train=True) -> Tuple[str, str, str, str]:
    folder = "train" if train else "test"
    data_path = os.path.join(ROOT_DIR, "data", folder, sector, activity)
    test_path = os.path.join(ROOT_DIR, "test", "results", "value_adding_analysis", folder, sector, activity)
    activity_breakdown_path = os.path.join(data_path, f"{activity}_activity_breakdown.json")
    ground_truth_path = os.path.join(data_path, f"{activity}_step_value_analysis.json")
    response_path = os.path.join(test_path, f"{model_name}_response.json")
    return test_path, activity_breakdown_path, ground_truth_path, response_path


In [2]:
def classify_value_adding(sector: str, activity: str, model: ValueClassificationComponentsGPT, model_name, train) -> None:
    logging.info(f"Processing sector: {sector}, activity: {activity}")
    
    try:
        # Load activity breakdown and ground truth
        test_path, activity_breakdown_path, ground_truth_path, response_path = get_file_paths(sector, activity, model_name, train=train)
        activity_breakdown: ActivityBreakdown = ActivityBreakdown.from_json(activity_breakdown_path)
        # ground_truth = ValueAddingAnalysis.from_json(ground_truth_path)
        
        activity_breakdown_dict = activity_breakdown.to_dict()
        
        logging.info(f"Processing with model: {model_name}")
        
        # Ensure test directory exists
        os.makedirs(test_path, exist_ok=True)
        
        # Get model response
        response = model.value_classification_step_level(activity_breakdown_dict)
        
        # Process model response
        model_value_adding_analysis: ValueAddingAnalysis = ValueAddingAnalysis.from_json(activity_breakdown_path)
        model_value_adding_analysis.classify_substeps(response)
        
        response_dict = {
            "model": {
                "name": model_name,
                "components": model.prompt_components["_raw_input"]
            },
            "response": model_value_adding_analysis.to_dict(),
            # "metrics": comparison_metrics.to_dict()
        }
        if not os.path.exists(test_path):
            os.makedirs(test_path)
        
        # Save results
        # with open(response_path, 'w') as f:
        with open(response_path, 'w') as f:
            json.dump(response_dict, f, indent=4)
        
        logging.info(f"Successfully processed and saved results for {sector}/{activity} with {model_name}")
    
    except Exception as e:
        logging.error(f"Error processing {sector}/{activity}: {str(e)}")
        
def save_model_configurations(models: Dict[str, ValueClassificationComponentsGPT], llm_version: str):
    """
    Save the configuration of each model as a JSON file.
    
    :param models: Dictionary of model names and their corresponding ValueClassificationComponentsGPT instances
    :param llm_version: Version of the LLM being used (e.g., "gpt-4")
    """
    output_dir = os.path.join("test", "models")
    os.makedirs(output_dir, exist_ok=True)
    
    for model_name, model in models.items():
        config = {
            "model_name": model_name,
            "prompt_components": model.prompt_components,
            "llm_version": llm_version
        }
        
        file_name = f"{model_name}.json"
        file_path = os.path.join(output_dir, file_name)
        
        with open(file_path, 'w') as f:
            json.dump(config, f, indent=2)
        
        print(f"Saved configuration for {model_name} to {file_path}")

In [3]:
models = [
        {
            "name": "GPT-3.5-Value-Classification-Neutral_Analyst_Basic",
            "components": VAPromptComponents.from_dict({
                "role_description": "neutral_analyst",
                "task_description": "standard",
                "classification_types": "basic",
                "function_definition": "basic",
                "parsing_instructions": "sequential",
                "output_format": "basic"
            })
        },
        {
            "name": "GPT-3.5-Value-Classification-Lean_Expert_Detailed",
            "components": VAPromptComponents.from_dict({
                "role_description": "lean_expert",
                "task_description": "efficiency_focused",
                "classification_types": "detailed",
                "function_definition": "detailed",
                "parsing_instructions": "holistic",
                "output_format": "structured",
                "additional_guidelines": "lean_principles"
            })
        },
        {
            "name": "GPT-3.5-Value-Classification-Process_Engineer_Technical",
            "components": VAPromptComponents.from_dict({
                "role_description": "process_engineer",
                "task_description": "standard",
                "classification_types": "detailed",
                "function_definition": "basic",
                "parsing_instructions": "sequential",
                "output_format": "basic",
                "example_output": "complex_process"
            })
        },
        {
            "name": "GPT-3.5-Value-Classification-Customer_Advocate_ValueFocused",
            "components": VAPromptComponents.from_dict({
                "role_description": "customer_advocate",
                "task_description": "efficiency_focused",
                "classification_types": "basic",
                "function_definition": "detailed",
                "parsing_instructions": "holistic",
                "output_format": "structured",
                "additional_guidelines": "context_aware"
            })
        },
        {
            "name": "GPT-3.5-Value-Classification-Business_Consultant_Strategic",
            "components": VAPromptComponents.from_dict({
                "role_description": "business_consultant",
                "task_description": "standard",
                "classification_types": "detailed",
                "function_definition": "detailed",
                "parsing_instructions": "holistic",
                "output_format": "structured",
                "example_output": "complex_process",
                "additional_guidelines": "context_aware"
            })
        },
        {
            "name": "GPT-3.5-Value-Classification-Neutral_Analyst_Comprehensive",
            "components": VAPromptComponents.from_dict({
                "role_description": "neutral_analyst",
                "task_description": "efficiency_focused",
                "classification_types": "detailed",
                "function_definition": "detailed",
                "parsing_instructions": "holistic",
                "output_format": "structured",
                "example_output": "complex_process",
                "additional_guidelines": "lean_principles"
            })
        },
        {
            "name": "GPT-3.5-Value-Classification-Lean_Expert_Minimal",
            "components": VAPromptComponents.from_dict({
                "role_description": "lean_expert",
                "task_description": "standard",
                "classification_types": "basic",
                "function_definition": "basic",
                "parsing_instructions": "sequential",
                "output_format": "basic"
            })
        },
        {
            "name": "GPT-3.5-Value-Classification-Process_Engineer_Detailed",
            "components": VAPromptComponents.from_dict({
                "role_description": "process_engineer",
                "task_description": "efficiency_focused",
                "classification_types": "detailed",
                "function_definition": "detailed",
                "parsing_instructions": "holistic",
                "output_format": "structured",
                "example_output": "complex_process",
                "additional_guidelines": "context_aware"
            })
        }
    ]

model = models[0]

value_classification_gpt = ValueClassificationComponentsGPT(model["components"])

In [5]:
TRAIN = False
model_idxs = [1, 2, 3, 4, 5]

def main():
    for model_idx in model_idxs:
        model = models[model_idx]
        sectors = get_sectors(train=TRAIN)
        for sector in sectors:
            activities = get_activities(sector, train=TRAIN)
            for activity in activities:
                classify_value_adding(sector, activity, value_classification_gpt, model["name"], train=TRAIN)

if __name__ == "__main__":
    main()

2024-10-22 12:10:09,210 - INFO - Processing sector: banking, activity: credibility_check
2024-10-22 12:10:09,224 - INFO - Processing with model: GPT-3.5-Value-Classification-Lean_Expert_Detailed
2024-10-22 12:10:13,428 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:10:18,228 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:10:23,782 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:10:23,800 - INFO - Successfully processed and saved results for banking/credibility_check with GPT-3.5-Value-Classification-Lean_Expert_Detailed
2024-10-22 12:10:23,800 - INFO 

2024-10-22 12:11:51,780 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:11:56,223 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:11:56,241 - INFO - Successfully processed and saved results for customer_service/incident_management with GPT-3.5-Value-Classification-Lean_Expert_Detailed
2024-10-22 12:11:56,241 - INFO - Processing sector: customer_service, activity: procure_parts
2024-10-22 12:11:56,241 - INFO - Processing with model: GPT-3.5-Value-Classification-Lean_Expert_Detailed
2024-10-22 12:11:56,241 - ERROR - Error processing customer_service/procure_parts: division by zero
2024-10-22 12:11:56,241 - INFO - Processing sector: customer_service, activity: service_request
2024-10-22 12:11:56,259 

2024-10-22 12:12:13,236 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:12:17,456 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:12:17,477 - INFO - Successfully processed and saved results for healthcare/register_patient with GPT-3.5-Value-Classification-Lean_Expert_Detailed
2024-10-22 12:12:17,477 - INFO - Processing sector: healthcare, activity: treatment_chamber
2024-10-22 12:12:17,482 - INFO - Processing with model: GPT-3.5-Value-Classification-Lean_Expert_Detailed
2024-10-22 12:12:24,145 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:12:24,145 - 

2024-10-22 12:12:38,444 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:12:43,209 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:12:43,215 - INFO - Successfully processed and saved results for technology/it_development with GPT-3.5-Value-Classification-Lean_Expert_Detailed
2024-10-22 12:12:43,216 - INFO - Processing sector: technology, activity: ticket_system
2024-10-22 12:12:43,227 - INFO - Processing with model: GPT-3.5-Value-Classification-Lean_Expert_Detailed
2024-10-22 12:12:48,011 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:12:52,390 - INFO -

2024-10-22 12:14:43,031 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:14:47,562 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:14:47,568 - INFO - Successfully processed and saved results for customer_service/incident_management with GPT-3.5-Value-Classification-Process_Engineer_Technical
2024-10-22 12:14:47,569 - INFO - Processing sector: customer_service, activity: procure_parts
2024-10-22 12:14:47,572 - INFO - Processing with model: GPT-3.5-Value-Classification-Process_Engineer_Technical
2024-10-22 12:14:47,574 - ERROR - Error processing customer_service/procure_parts: division by zero
2024-10-22 12:14:47,575 - INFO - Processing sector: customer_service, activity: service_request
2024-10-22 1

...

2024-10-22 12:19:44,726 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:19:48,454 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:19:48,467 - INFO - Successfully processed and saved results for healthcare/register_patient with GPT-3.5-Value-Classification-Business_Consultant_Strategic
2024-10-22 12:19:48,469 - INFO - Processing sector: healthcare, activity: treatment_chamber
2024-10-22 12:19:48,489 - INFO - Processing with model: GPT-3.5-Value-Classification-Business_Consultant_Strategic
2024-10-22 12:19:53,123 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-

2024-10-22 12:20:07,225 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:20:13,677 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:20:13,692 - INFO - Successfully processed and saved results for technology/it_development with GPT-3.5-Value-Classification-Business_Consultant_Strategic
2024-10-22 12:20:13,693 - INFO - Processing sector: technology, activity: ticket_system
2024-10-22 12:20:13,709 - INFO - Processing with model: GPT-3.5-Value-Classification-Business_Consultant_Strategic
2024-10-22 12:20:19,269 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:

2024-10-22 12:21:53,674 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:21:57,960 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:21:57,969 - INFO - Successfully processed and saved results for customer_service/incident_management with GPT-3.5-Value-Classification-Neutral_Analyst_Comprehensive
2024-10-22 12:21:57,971 - INFO - Processing sector: customer_service, activity: procure_parts
2024-10-22 12:21:57,974 - INFO - Processing with model: GPT-3.5-Value-Classification-Neutral_Analyst_Comprehensive
2024-10-22 12:21:57,977 - ERROR - Error processing customer_service/procure_parts: division by zero
2024-10-22 12:21:57,978 - INFO - Processing sector: customer_service, activity: service_request
2024-1

2024-10-22 12:22:15,644 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:22:19,614 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:22:19,619 - INFO - Successfully processed and saved results for healthcare/register_patient with GPT-3.5-Value-Classification-Neutral_Analyst_Comprehensive
2024-10-22 12:22:19,620 - INFO - Processing sector: healthcare, activity: treatment_chamber
2024-10-22 12:22:19,623 - INFO - Processing with model: GPT-3.5-Value-Classification-Neutral_Analyst_Comprehensive
2024-10-22 12:22:26,823 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-

2024-10-22 12:22:41,587 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:22:45,626 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12:22:45,631 - INFO - Successfully processed and saved results for technology/it_development with GPT-3.5-Value-Classification-Neutral_Analyst_Comprehensive
2024-10-22 12:22:45,631 - INFO - Processing sector: technology, activity: ticket_system
2024-10-22 12:22:45,633 - INFO - Processing with model: GPT-3.5-Value-Classification-Neutral_Analyst_Comprehensive
2024-10-22 12:22:50,554 - INFO - HTTP Request: POST https://datascience-openai-local.openai.azure.com//openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
2024-10-22 12: